In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 16:52:37.544222 140277997516608 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 16:52:37.553567 140277997516608 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 16:52:37.559082 140277997516608 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 16:52:37.565951 140277997516608 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 16:52:37.571066 140277997516608 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 16:52:37.579940 140277997516608 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 16:52:37.622932 140277997516608 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 16:52:37.702895 140277997516608 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9320 train_acc= 0.3857 val_loss= 1.9337 val_acc= 0.3833 time= 1.0840
Epoch: 0002 train_loss= 1.9169 train_acc= 0.3214 val_loss= 1.9200 val_acc= 0.3533 time= 0.0235
Epoch: 0003 train_loss= 1.9003 train_acc= 0.3357 val_loss= 1.9057 val_acc= 0.3600 time= 0.0202
Epoch: 0004 train_loss= 1.8826 train_acc= 0.3857 val_loss= 1.8908 val_acc= 0.3633 time= 0.0264
Epoch: 0005 train_loss= 1.8641 train_acc= 0.3929 val_loss= 1.8754 val_acc= 0.3667 time= 0.0245
Epoch: 0006 train_loss= 1.8454 train_acc= 0.4143 val_loss= 1.8600 val_acc= 0.3700 time= 0.0245
Epoch: 0007 train_loss= 1.8266 train_acc= 0.4143 val_loss= 1.8447 val_acc= 0.3767 time= 0.0187
Epoch: 0008 train_loss= 1.8078 train_acc= 0.4214 val_loss= 1.8296 val_acc= 0.3867 time= 0.0227
Epoch: 0009 train_loss= 1.7895 train_acc= 0.4286 val_loss= 1.8151 val_acc= 0.3967 time= 0.0286
Epoch: 0010 train_loss= 1.7718 train_acc= 0.4286 val_loss= 1.8013 val_acc= 0.4000 time= 0.0255
Epoch: 0011 train_loss= 1.7550 train_acc= 0.4286 v

Epoch: 0089 train_loss= 0.7586 train_acc= 0.8857 val_loss= 1.0415 val_acc= 0.7733 time= 0.0239
Epoch: 0090 train_loss= 0.7508 train_acc= 0.8857 val_loss= 1.0369 val_acc= 0.7767 time= 0.0229
Epoch: 0091 train_loss= 0.7433 train_acc= 0.9000 val_loss= 1.0327 val_acc= 0.7867 time= 0.0187
Epoch: 0092 train_loss= 0.7365 train_acc= 0.9071 val_loss= 1.0287 val_acc= 0.7833 time= 0.0197
Epoch: 0093 train_loss= 0.7296 train_acc= 0.9000 val_loss= 1.0236 val_acc= 0.7867 time= 0.0189
Epoch: 0094 train_loss= 0.7231 train_acc= 0.9000 val_loss= 1.0181 val_acc= 0.7867 time= 0.0238
Epoch: 0095 train_loss= 0.7167 train_acc= 0.9000 val_loss= 1.0122 val_acc= 0.7867 time= 0.0199
Epoch: 0096 train_loss= 0.7104 train_acc= 0.9000 val_loss= 1.0053 val_acc= 0.7867 time= 0.0186
Epoch: 0097 train_loss= 0.7044 train_acc= 0.9143 val_loss= 0.9992 val_acc= 0.7900 time= 0.0201
Epoch: 0098 train_loss= 0.6984 train_acc= 0.9143 val_loss= 0.9937 val_acc= 0.7867 time= 0.0234
Epoch: 0099 train_loss= 0.6927 train_acc= 0.9143 v

Epoch: 0183 train_loss= 0.4065 train_acc= 0.9714 val_loss= 0.7718 val_acc= 0.8033 time= 0.0235
Epoch: 0184 train_loss= 0.4050 train_acc= 0.9714 val_loss= 0.7709 val_acc= 0.8033 time= 0.0230
Epoch: 0185 train_loss= 0.4018 train_acc= 0.9714 val_loss= 0.7701 val_acc= 0.8033 time= 0.0253
Epoch: 0186 train_loss= 0.3990 train_acc= 0.9714 val_loss= 0.7701 val_acc= 0.8067 time= 0.0228
Epoch: 0187 train_loss= 0.3969 train_acc= 0.9714 val_loss= 0.7690 val_acc= 0.8067 time= 0.0232
Epoch: 0188 train_loss= 0.3954 train_acc= 0.9714 val_loss= 0.7691 val_acc= 0.8067 time= 0.0222
Epoch: 0189 train_loss= 0.3946 train_acc= 0.9643 val_loss= 0.7693 val_acc= 0.8133 time= 0.0237
Epoch: 0190 train_loss= 0.3934 train_acc= 0.9643 val_loss= 0.7704 val_acc= 0.8200 time= 0.0229
Epoch: 0191 train_loss= 0.3917 train_acc= 0.9714 val_loss= 0.7701 val_acc= 0.8200 time= 0.0249
Epoch: 0192 train_loss= 0.3889 train_acc= 0.9714 val_loss= 0.7678 val_acc= 0.8100 time= 0.0236
Epoch: 0193 train_loss= 0.3860 train_acc= 0.9714 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.7835
accuracy = 0.8130


# Gaussian Graph Convolution

In [15]:
H = Dropout(0.5)(X_in)
H = GaussianGraphConvolution(16,
    is_first=True, 
    activation='relu', 
#     kernel_regularizer=l2(5e-4)
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax'
)(H+G)

W1110 16:52:43.255880 140277997516608 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:153: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 16:52:43.256954 140277997516608 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should updat

In [16]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [18]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9410 train_acc= 0.3643 val_loss= 1.9428 val_acc= 0.2800 time= 0.4193
Epoch: 0002 train_loss= 1.9348 train_acc= 0.4571 val_loss= 1.9377 val_acc= 0.3800 time= 0.0237
Epoch: 0003 train_loss= 1.9259 train_acc= 0.5143 val_loss= 1.9307 val_acc= 0.4633 time= 0.0242
Epoch: 0004 train_loss= 1.9142 train_acc= 0.6000 val_loss= 1.9236 val_acc= 0.4800 time= 0.0228
Epoch: 0005 train_loss= 1.9039 train_acc= 0.5714 val_loss= 1.9140 val_acc= 0.4867 time= 0.0251
Epoch: 0006 train_loss= 1.8915 train_acc= 0.6143 val_loss= 1.9080 val_acc= 0.4933 time= 0.0206
Epoch: 0007 train_loss= 1.8814 train_acc= 0.6571 val_loss= 1.8976 val_acc= 0.4967 time= 0.0242
Epoch: 0008 train_loss= 1.8684 train_acc= 0.6286 val_loss= 1.8916 val_acc= 0.4867 time= 0.0230
Epoch: 0009 train_loss= 1.8532 train_acc= 0.6857 val_loss= 1.8783 val_acc= 0.5267 time= 0.0183
Epoch: 0010 train_loss= 1.8382 train_acc= 0.6857 val_loss= 1.8669 val_acc= 0.5633 time= 0.0234
Epoch: 0011 train_loss= 1.8230 train_acc= 0.6929 v

Epoch: 0088 train_loss= 0.3431 train_acc= 0.9714 val_loss= 0.7405 val_acc= 0.8233 time= 0.0232
Epoch: 0089 train_loss= 0.3345 train_acc= 0.9714 val_loss= 0.7343 val_acc= 0.8233 time= 0.0238
Epoch: 0090 train_loss= 0.3264 train_acc= 0.9714 val_loss= 0.7301 val_acc= 0.8233 time= 0.0192
Epoch: 0091 train_loss= 0.3179 train_acc= 0.9714 val_loss= 0.7254 val_acc= 0.8233 time= 0.0189
Epoch: 0092 train_loss= 0.3093 train_acc= 0.9714 val_loss= 0.7199 val_acc= 0.8233 time= 0.0194
Epoch: 0093 train_loss= 0.3021 train_acc= 0.9786 val_loss= 0.7151 val_acc= 0.8233 time= 0.0234
Epoch: 0094 train_loss= 0.2931 train_acc= 0.9786 val_loss= 0.7094 val_acc= 0.8233 time= 0.0235
Epoch: 0095 train_loss= 0.2860 train_acc= 0.9786 val_loss= 0.7065 val_acc= 0.8300 time= 0.0232
Epoch: 0096 train_loss= 0.2780 train_acc= 0.9786 val_loss= 0.7013 val_acc= 0.8267 time= 0.0228
Epoch: 0097 train_loss= 0.2719 train_acc= 0.9786 val_loss= 0.6988 val_acc= 0.8267 time= 0.0190
Epoch: 0098 train_loss= 0.2655 train_acc= 0.9786 v

In [19]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.6204
accuracy = 0.8160


# Attacks